In [1]:
import sys

sys.path.append("..")

In [2]:
%matplotlib inline

In [3]:
import numpy as np
import pandas as pd

from src.const import EventData, GameData, GameBoxScoreData, GameSummaryData, TeamData
from src.data import load
from src.utils import calc_posession

- ベンチスコア平均（チーム）
- 4Qスコア平均（チーム）
    - 点差5点差以内、4Q平均試投数・平均得点（チーム）
- 3P試投数・成功数・成功率（選手・チーム）
- FT試投数・成功数・成功率（選手・チーム）
- TO / posession（選手・チーム）
- Offence Efficiency（チーム）
- Deffence Efficiency（チーム）
    - 去年と今年の比較

In [4]:
df_game = load(GameData.games_202021)
display(df_game.head().T)

,0,1,2,3,4
ScheduleKey,5938,5939,5890,5903,5891
Season,2020-21,2020-21,2020-21,2020-21,2020-21
EventId,2,2,2,2,2
Date,2020.10.24,2020.10.25,2020.10.09,2020.10.18,2020.10.10
Arena,おおきにアリーナ舞洲,おおきにアリーナ舞洲,広島サンプラザホール,横浜国際プール,広島サンプラザホール
Attendance,1675,1255,2061,1517,2221
HomeTeamId,700,700,721,694,721
AwayTeamId,706,706,706,706,706


In [5]:
df_game_box = load(GameBoxScoreData.games_boxscore_202021)
display(df_game_box.head().T)

,0,1,2,3,4
ScheduleKey,5938,5938,5938,5938,5938
TeamId,700,700,700,700,700
BoxType,Total,Total,Total,Total,Total
PlayerId,10849,32984,32982,32983,30349
Player,伊藤達哉,エリエット・ドンリー,角野亮伍,駒水大雅ジャック,中村浩陸
Number,2,3,8,10,13
Position,PG,SF,SF,SF,PG
StarterBench,Starter,Bench,Starter,Bench,Bench
MIN,21.9,1.93,31.65,1.4,0
MIN.STR,21:54,01:56,31:39,01:24,00:00


In [6]:
df_game_summary = load(GameSummaryData.games_summary_202021)
display(df_game_summary.head().T)

,0,1,2,3,4
ScheduleKey,5938,5938,5939,5939,5890
TeamId,700,706,700,706,721
PTS,69,97,77,85,71
Q1,21,23,15,29,21
Q2,8,27,23,14,16
Q3,21,31,16,17,18
Q4,19,16,23,25,16
OT1,0,0,0,0,0
OT2,0,0,0,0,0
OT3,0,0,0,0,0


In [7]:
df_team = load(TeamData.teams)
df_team = df_team[(df_team["Season"]=="2020-21") & (df_team["League"]=="B1")]
display(df_team.head().T)

,144,145,146,147,148
TeamId,702,693,703,704,706
Season,2020-21,2020-21,2020-21,2020-21,2020-21
League,B1,B1,B1,B1,B1
NameShort,北海道,秋田,宇都宮,千葉,A東京
NameLong,レバンガ北海道,秋田ノーザンハピネッツ,宇都宮ブレックス,千葉ジェッツ,アルバルク東京
Division,東地区,東地区,東地区,東地区,東地区


## ベンチスコア

In [8]:
df_bench = df_game_box[df_game_box["StarterBench"]=="Bench"].groupby("TeamId").agg({"MIN": ["sum"], "PTS": ["sum"], "EFF": ["mean"]})
df_bench.columns = [f"{c[0]}_{c[1]}" for c in df_bench.columns]
df_bench = df_bench.merge(df_team.loc[:, ["TeamId", "NameShort", "Division"]], how="inner", on="TeamId")

In [9]:
df_bench.sort_values("PTS_sum", ascending=False)

,TeamId,MIN_sum,PTS_sum,EFF_mean,NameShort,Division
10,703,1322.98,509,5.200000,宇都宮,東地区
12,706,1209.11,461,4.653846,A東京,東地区
14,720,1213.92,438,4.371429,島根,西地区
16,726,1372.83,430,3.933333,SR渋谷,東地区
17,727,1234.71,428,4.526316,川崎,東地区
9,702,1143.67,406,3.244898,北海道,東地区
18,728,1101.19,400,4.647059,三河,西地区
15,721,1124.57,400,3.923810,広島,西地区
7,700,851.27,393,4.915663,大阪,西地区
11,704,975.35,378,4.811111,千葉,東地区


In [10]:
df_bench.sort_values("EFF_mean", ascending=False)

,TeamId,MIN_sum,PTS_sum,EFF_mean,NameShort,Division
10,703,1322.98,509,5.200000,宇都宮,東地区
7,700,851.27,393,4.915663,大阪,西地区
11,704,975.35,378,4.811111,千葉,東地区
12,706,1209.11,461,4.653846,A東京,東地区
18,728,1101.19,400,4.647059,三河,西地区
17,727,1234.71,428,4.526316,川崎,東地区
14,720,1213.92,438,4.371429,島根,西地区
16,726,1372.83,430,3.933333,SR渋谷,東地区
15,721,1124.57,400,3.923810,広島,西地区
0,693,1008.11,358,3.415730,秋田,東地区


## 4Qスコア平均

In [11]:
df_q = df_game_summary.groupby("TeamId").agg({"Q1": ["mean"], "Q4": ["mean"], "PTS": ["mean"]})
df_q.columns = [f"{c[0]}_{c[1]}" for c in df_q.columns]
df_q["Q1_PTS_rate"] = df_q["Q1_mean"] / df_q["PTS_mean"]
df_q["Q4_PTS_rate"] = df_q["Q4_mean"] / df_q["PTS_mean"]
df_q = df_q.merge(df_team.loc[:, ["TeamId", "NameShort", "Division"]], how="inner", on="TeamId")

In [12]:
df_q.sort_values("Q4_mean", ascending=False)

,TeamId,Q1_mean,Q4_mean,PTS_mean,Q1_PTS_rate,Q4_PTS_rate,NameShort,Division
5,698,18.066667,23.666667,77.466667,0.233219,0.305508,滋賀,西地区
17,727,20.533333,22.800000,81.600000,0.251634,0.279412,川崎,東地区
18,728,22.400000,22.666667,83.266667,0.269015,0.272218,三河,西地区
3,696,23.133333,22.400000,89.133333,0.259536,0.251309,富山,東地区
16,726,20.533333,22.333333,82.000000,0.250407,0.272358,SR渋谷,東地区
11,704,20.733333,22.266667,90.133333,0.230030,0.247041,千葉,東地区
10,703,21.000000,22.000000,82.533333,0.254443,0.266559,宇都宮,東地区
2,695,17.266667,21.800000,75.200000,0.229610,0.289894,新潟,東地区
9,702,20.000000,21.200000,80.666667,0.247934,0.262810,北海道,東地区
8,701,18.200000,20.933333,78.866667,0.230769,0.265427,琉球,西地区


In [13]:
df_q.sort_values("Q4_PTS_rate", ascending=False)

,TeamId,Q1_mean,Q4_mean,PTS_mean,Q1_PTS_rate,Q4_PTS_rate,NameShort,Division
5,698,18.066667,23.666667,77.466667,0.233219,0.305508,滋賀,西地区
2,695,17.266667,21.800000,75.200000,0.229610,0.289894,新潟,東地区
17,727,20.533333,22.800000,81.600000,0.251634,0.279412,川崎,東地区
16,726,20.533333,22.333333,82.000000,0.250407,0.272358,SR渋谷,東地区
18,728,22.400000,22.666667,83.266667,0.269015,0.272218,三河,西地区
4,697,15.466667,19.133333,71.533333,0.216216,0.267474,三遠,西地区
10,703,21.000000,22.000000,82.533333,0.254443,0.266559,宇都宮,東地区
8,701,18.200000,20.933333,78.866667,0.230769,0.265427,琉球,西地区
9,702,20.000000,21.200000,80.666667,0.247934,0.262810,北海道,東地区
19,729,17.200000,20.533333,78.333333,0.219574,0.262128,名古屋D,西地区


In [14]:
df_q.sort_values("Q1_mean", ascending=False)

,TeamId,Q1_mean,Q4_mean,PTS_mean,Q1_PTS_rate,Q4_PTS_rate,NameShort,Division
3,696,23.133333,22.400000,89.133333,0.259536,0.251309,富山,東地区
18,728,22.400000,22.666667,83.266667,0.269015,0.272218,三河,西地区
10,703,21.000000,22.000000,82.533333,0.254443,0.266559,宇都宮,東地区
11,704,20.733333,22.266667,90.133333,0.230030,0.247041,千葉,東地区
16,726,20.533333,22.333333,82.000000,0.250407,0.272358,SR渋谷,東地区
17,727,20.533333,22.800000,81.600000,0.251634,0.279412,川崎,東地区
9,702,20.000000,21.200000,80.666667,0.247934,0.262810,北海道,東地区
15,721,19.333333,19.000000,78.266667,0.247019,0.242760,広島,西地区
14,720,18.800000,20.333333,78.533333,0.239389,0.258913,島根,西地区
7,700,18.533333,20.400000,79.666667,0.232636,0.256067,大阪,西地区


In [15]:
df_q.sort_values("Q1_PTS_rate", ascending=False)

,TeamId,Q1_mean,Q4_mean,PTS_mean,Q1_PTS_rate,Q4_PTS_rate,NameShort,Division
18,728,22.400000,22.666667,83.266667,0.269015,0.272218,三河,西地区
3,696,23.133333,22.400000,89.133333,0.259536,0.251309,富山,東地区
10,703,21.000000,22.000000,82.533333,0.254443,0.266559,宇都宮,東地区
13,716,17.533333,17.200000,69.133333,0.253616,0.248795,信州,西地区
17,727,20.533333,22.800000,81.600000,0.251634,0.279412,川崎,東地区
16,726,20.533333,22.333333,82.000000,0.250407,0.272358,SR渋谷,東地区
9,702,20.000000,21.200000,80.666667,0.247934,0.262810,北海道,東地区
15,721,19.333333,19.000000,78.266667,0.247019,0.242760,広島,西地区
14,720,18.800000,20.333333,78.533333,0.239389,0.258913,島根,西地区
5,698,18.066667,23.666667,77.466667,0.233219,0.305508,滋賀,西地区


## 3PT

In [16]:
df_3pt = df_game_summary.groupby("TeamId").agg({"F3GM": ["sum"], "F3GA": ["sum"]})
df_3pt.columns = [f"{c[0]}_{c[1]}" for c in df_3pt.columns]
df_3pt["F3GP"] = df_3pt["F3GM_sum"] / df_3pt["F3GA_sum"]
df_3pt = df_3pt.merge(df_team.loc[:, ["TeamId", "NameShort", "Division"]], how="inner", on="TeamId")

In [17]:
df_3pt.sort_values("F3GP", ascending=False)

,TeamId,F3GM_sum,F3GA_sum,F3GP,NameShort,Division
18,728,157,387,0.405685,三河,西地区
11,704,137,361,0.379501,千葉,東地区
19,729,132,351,0.376068,名古屋D,西地区
10,703,129,346,0.372832,宇都宮,東地区
9,702,138,375,0.368000,北海道,東地区
12,706,116,320,0.362500,A東京,東地区
1,694,112,318,0.352201,横浜,東地区
5,698,144,412,0.349515,滋賀,西地区
3,696,123,353,0.348442,富山,東地区
8,701,104,304,0.342105,琉球,西地区


In [18]:
df_3pt.sort_values("F3GA_sum", ascending=False)

,TeamId,F3GM_sum,F3GA_sum,F3GP,NameShort,Division
13,716,139,437,0.318078,信州,西地区
5,698,144,412,0.349515,滋賀,西地区
17,727,123,401,0.306733,川崎,東地区
18,728,157,387,0.405685,三河,西地区
9,702,138,375,0.368000,北海道,東地区
16,726,116,367,0.316076,SR渋谷,東地区
11,704,137,361,0.379501,千葉,東地区
3,696,123,353,0.348442,富山,東地区
0,693,90,352,0.255682,秋田,東地区
19,729,132,351,0.376068,名古屋D,西地区


## FT

In [19]:
df_ft = df_game_summary.groupby("TeamId").agg({"FTM": ["sum"], "FTA": ["sum"]})
df_ft.columns = [f"{c[0]}_{c[1]}" for c in df_ft.columns]
df_ft["FTP"] = df_ft["FTM_sum"] / df_ft["FTA_sum"]
df_ft = df_ft.merge(df_team.loc[:, ["TeamId", "NameShort", "Division"]], how="inner", on="TeamId")

In [20]:
df_ft.sort_values("FTP", ascending=False)

,TeamId,FTM_sum,FTA_sum,FTP,NameShort,Division
2,695,211,264,0.799242,新潟,東地区
4,697,176,225,0.782222,三遠,西地区
15,721,225,289,0.778547,広島,西地区
9,702,192,247,0.777328,北海道,東地区
6,699,229,296,0.773649,京都,西地区
17,727,225,291,0.773196,川崎,東地区
12,706,185,241,0.767635,A東京,東地区
18,728,174,228,0.763158,三河,西地区
19,729,227,304,0.746711,名古屋D,西地区
13,716,188,253,0.743083,信州,西地区


In [21]:
df_ft = df_game_box.groupby("Player").agg({"FTM": ["sum"], "FTA": ["sum"]})
df_ft.columns = [f"{c[0]}_{c[1]}" for c in df_ft.columns]
df_ft["FTP"] = df_ft["FTM_sum"] / df_ft["FTA_sum"]
df_ft = df_ft[df_ft["FTA_sum"] > 20]

In [22]:
df_ft.sort_values("FTP", ascending=False).head(20)

,FTM_sum,FTA_sum,FTP
Player,,,
佐々木隆成,22,22,1.000000
畠山俊樹,20,21,0.952381
トーマス・ケネディ,27,29,0.931034
ギャレット・スタツ,23,25,0.920000
ステヴァン・イェロヴァツ,42,46,0.913043
木田貴明,37,41,0.902439
ジェフ・エアーズ,58,65,0.892308
相馬卓弥,24,27,0.888889
辻直人,31,35,0.885714


In [23]:
df_ft.sort_values("FTP", ascending=False).tail(20)

,FTM_sum,FTA_sum,FTP
Player,,,
小林遥太,13,22,0.590909
アブドゥーラ・クウソー,13,22,0.590909
カディーム・コールビー,40,68,0.588235
木村啓太郎,21,36,0.583333
シェーファーアヴィ幸樹,12,21,0.571429
小野龍猛,12,21,0.571429
ケビン・コッツァー,26,46,0.565217
ジョシュア・クロフォード,26,46,0.565217
チリジ・ネパウェ,39,69,0.565217


## TO / Possessions

In [24]:
df_game_summary["Posession"] = [
    calc_posession(F2GA + F3GA, TO, FTA, OR)
    for F2GA, F3GA, TO, FTA, OR 
    in zip(df_game_summary["F2GA"], df_game_summary["F3GA"], df_game_summary["TO"], df_game_summary["FTA"], df_game_summary["OR"])
]

In [25]:
df_game_summary["TO_per_Posession"] = 100 * df_game_summary["TO"] / df_game_summary["Posession"]

df_TO = df_game_summary.groupby("TeamId").agg({"TO_per_Posession": ["mean"]})
df_TO.columns = [f"{c[0]}_{c[1]}" for c in df_TO.columns]
df_TO = df_TO.merge(df_team.loc[:, ["TeamId", "NameShort", "Division"]], how="inner", on="TeamId")

In [26]:
df_TO.sort_values("TO_per_Posession_mean", ascending=True)

,TeamId,TO_per_Posession_mean,NameShort,Division
12,706,13.724524,A東京,東地区
9,702,14.558581,北海道,東地区
6,699,14.874169,京都,西地区
18,728,15.509072,三河,西地区
10,703,15.774837,宇都宮,東地区
17,727,15.857444,川崎,東地区
5,698,15.878544,滋賀,西地区
4,697,16.888618,三遠,西地区
15,721,17.098357,広島,西地区
3,696,17.267151,富山,東地区


In [43]:
df_game_box.head().T

,0,1,2,3,4
ScheduleKey,5938,5938,5938,5938,5938
TeamId,700,700,700,700,700
BoxType,Total,Total,Total,Total,Total
PlayerId,10849,32984,32982,32983,30349
Player,伊藤達哉,エリエット・ドンリー,角野亮伍,駒水大雅ジャック,中村浩陸
Number,2,3,8,10,13
Position,PG,SF,SF,SF,PG
StarterBench,Starter,Bench,Starter,Bench,Bench
MIN,21.9,1.93,31.65,1.4,0
MIN.STR,21:54,01:56,31:39,01:24,00:00


In [63]:
df_TO = df_game_box.groupby("Player").agg({"TO": ["mean", "sum"], "MIN": ["mean", "sum"]})
df_TO.columns = [f"{c[0]}_{c[1]}" for c in df_TO.columns]
df_TO = df_TO.merge(df_game_box.loc[:, ["Player", "Position"]].drop_duplicates(), how="left", on="Player")
df_TO = df_TO[df_TO["MIN_mean"] > 16]

In [64]:
df_TO.sort_values("TO_mean", ascending=False).head(20)

,Player,TO_mean,TO_sum,MIN_mean,MIN_sum,Position
98,テレンス・ウッドベリー,6.000000,12,27.350000,54.70,SF/PF
71,ジュリアン・マブンガ,4.066667,61,33.959333,509.39,PF
419,西山達哉,3.666667,55,28.680000,430.20,PG
72,ジョシュア・クロフォード,3.444444,31,34.283333,308.55,C
85,スティーブン・ハート,3.363636,37,35.567273,391.24,C/PF
188,伊藤達哉,3.200000,48,25.235333,378.53,PG
11,アンソニー・マクヘンリー,3.090909,34,34.692727,381.62,C/PF
150,リース・ヴァーグ,3.000000,15,33.572000,167.86,PF
378,石川海斗,2.923077,38,32.724615,425.42,PG
118,パトリック・アウダ,2.888889,26,25.906667,233.16,PF


In [65]:
df_TO.sort_values("TO_mean", ascending=False).tail(20)

,Player,TO_mean,TO_sum,MIN_mean,MIN_sum,Position
259,小野龍猛,0.615385,8,16.110769,209.44,SF
329,橋本竜馬,0.600000,9,20.798667,311.98,PG
386,秋山皓太,0.600000,9,17.851333,267.77,SG/SF
77,ジョシュ・ペッパーズ,0.600000,3,16.248000,81.24,SF/PF
413,葛原大智,0.600000,9,17.525333,262.88,SG/SF
332,比江島慎,0.600000,9,18.193333,272.90,SG
466,飯田遼,0.545455,6,18.662727,205.29,SG/SF
462,須田侑太郎,0.533333,8,20.300667,304.51,SG/SF
333,水戸健史,0.533333,8,16.145333,242.18,SG
204,前田悟,0.533333,8,29.412000,441.18,SF


In [66]:
df_TO_PG = df_TO[df_TO["Position"].isin(["PG", "PG/SG"])]

In [69]:
df_TO_PG.sort_values("TO_mean", ascending=False).head(20)

,Player,TO_mean,TO_sum,MIN_mean,MIN_sum,Position
419,西山達哉,3.666667,55,28.680000,430.20,PG
188,伊藤達哉,3.200000,48,25.235333,378.53,PG
378,石川海斗,2.923077,38,32.724615,425.42,PG
479,齋藤拓実,2.800000,42,24.168000,362.52,PG
396,納見悠仁,2.800000,42,24.454667,366.82,PG/SG
237,宇都直輝,2.733333,41,27.302000,409.53,PG
201,兒玉貴通,2.545455,28,29.494545,324.44,PG
80,ジョーダン・テイラー,2.333333,28,19.545000,234.54,PG
322,楯昌宗,2.300000,23,19.193000,191.93,PG
116,ネナド・ミリェノヴィッチ,2.250000,9,21.392500,85.57,PG


In [70]:
df_TO_PG.sort_values("TO_mean", ascending=False).tail(20)

,Player,TO_mean,TO_sum,MIN_mean,MIN_sum,Position
176,久岡幸太郎,1.181818,13,18.055455,198.61,PG
294,木村啓太郎,1.166667,14,18.247500,218.97,PG
454,長谷川暢,1.153846,15,19.986923,259.83,PG
239,安藤誓哉,1.133333,17,23.802000,357.03,PG
353,狩俣昌也,1.133333,17,24.406000,366.09,PG
275,岸本隆一,1.133333,17,21.414667,321.22,PG/SG
381,神原裕司,1.090909,12,23.674545,260.42,PG/SG
192,佐々木隆成,1.076923,14,24.024615,312.32,PG/SG
434,遠藤祐亮,1.066667,16,22.453333,336.80,PG/SG
301,村上駿斗,1.000000,15,21.656000,324.84,PG/SG


## Offence Efficiency

In [27]:
df_game_summary["PTS_per_Posession"] = 100 * df_game_summary["PTS"] / df_game_summary["Posession"]

In [28]:
df_OE = df_game_summary.groupby("TeamId").agg({"PTS_per_Posession": ["mean"]})
df_OE.columns = [f"{c[0]}_{c[1]}" for c in df_OE.columns]
df_OE = df_OE.merge(df_team.loc[:, ["TeamId", "NameShort", "Division"]], how="inner", on="TeamId")

In [29]:
df_OE.sort_values("PTS_per_Posession_mean", ascending=False)

,TeamId,PTS_per_Posession_mean,NameShort,Division
11,704,129.200988,千葉,東地区
3,696,121.441819,富山,東地区
18,728,121.126116,三河,西地区
10,703,120.112997,宇都宮,東地区
12,706,117.789918,A東京,東地区
17,727,116.591838,川崎,東地区
9,702,114.495321,北海道,東地区
16,726,114.310069,SR渋谷,東地区
19,729,113.870096,名古屋D,西地区
8,701,113.865789,琉球,西地区


## Defence Efficiency

In [30]:
pts_opponent = {
    k: df_group["PTS"].values.tolist()
    for k, df_group 
    in df_game_summary.loc[:, ["ScheduleKey", "TeamId", "PTS"]].groupby("ScheduleKey")
}
df_game_summary["PTS_opponent"] = df_game_summary["ScheduleKey"].apply(lambda x: pts_opponent.get(x))
df_game_summary["PTS_opponent"] = [
    pts_list[0] if i % 2 else pts_list[1]
    for i, pts_list
    in enumerate(df_game_summary["PTS_opponent"])
]

df_game_summary["PTS_opponent_per_Posession"] = 100 * df_game_summary["PTS_opponent"] / df_game_summary["Posession"]

In [31]:
df_DE = df_game_summary.groupby("TeamId").agg({"PTS_opponent_per_Posession": ["mean"]})
df_DE.columns = [f"{c[0]}_{c[1]}" for c in df_DE.columns]
df_DE = df_DE.merge(df_team.loc[:, ["TeamId", "NameShort", "Division"]], how="inner", on="TeamId")

In [32]:
df_DE.sort_values("PTS_opponent_per_Posession_mean", ascending=True)

,TeamId,PTS_opponent_per_Posession_mean,NameShort,Division
10,703,98.826270,宇都宮,東地区
0,693,104.755273,秋田,東地区
8,701,105.767344,琉球,西地区
11,704,106.203095,千葉,東地区
17,727,106.500459,川崎,東地区
12,706,107.513825,A東京,東地区
3,696,107.980424,富山,東地区
19,729,108.629577,名古屋D,西地区
7,700,111.683792,大阪,西地区
13,716,112.600560,信州,西地区
